In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-tv-shows-and-movie-list/netflix_list.csv


In [2]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

****data collection and preprocessing****

In [3]:
#loading the file
movie_data=pd.read_csv('../input/netflix-tv-shows-and-movie-list/netflix_list.csv')


In [4]:
movie_data['index']= range(0,len(movie_data))
#movie_data
#movie_data.drop(['index_name'],axis=1,inplace=True)
#m=range(1,3)
movie_data.insert(0,'index_1','')
movie_data['index_1']=movie_data['index']
movie_data.drop(['index'],axis=1,inplace=True)
movie_data.rename({'index_1':'index'},axis=1,inplace=True)



In [5]:
movie_data


,index,imdb_id,title,popular_rank,certificate,startYear,endYear,episodes,runtime,type,orign_country,language,plot,summary,rating,numVotes,genres,isAdult,cast,image_url
0,0,tt4052886,Lucifer,1,15,2016.0,NaN,93.0,42,tvSeries,United States,English,Lucifer Morningstar has decided he's had enoug...,"Lucifer Morningstar, bored from his sulking li...",8.1,250884.0,"Crime,Drama,Fantasy",0,"['Tom Ellis', 'Lauren German', 'Lesley-Ann Bra...",https://m.media-amazon.com/images/M/MV5BNzY1Yj...
1,1,tt0993840,Army of the Dead,2,18,2021.0,NaN,NaN,148,movie,United States,English,"Following a zombie outbreak in Las Vegas, a gr...","With the abandoned, walled city of Las Vegas o...",5.8,110780.0,"Action,Crime,Horror",0,"['Dave Bautista', 'Ella Purnell', 'Ana de la R...",https://m.media-amazon.com/images/M/MV5BNGY0Nz...
2,2,tt7255502,The Kominsky Method,3,18,2018.0,2021.0,22.0,30,tvSeries,United States,English,"An aging actor, who long ago enjoyed a brush w...",Michael Douglas plays an actor who made it big...,8.2,28795.0,"Comedy,Drama",0,"['Michael Douglas', 'Sarah Baker', 'Graham Rog...",https://m.media-amazon.com/images/M/MV5BMzA0YT...
3,3,tt0108778,Friends,4,13+,1994.0,2004.0,235.0,22,tvSeries,United States,English,Follows the personal and professional lives of...,"Ross Geller, Rachel Green, Monica Geller, Joey...",8.9,861843.0,"Comedy,Romance",0,"['Jennifer Aniston', 'Courteney Cox', 'Lisa Ku...",https://m.media-amazon.com/images/M/MV5BNDVkYj...
4,4,tt9251798,Ragnarok,5,18,2020.0,NaN,12.0,45,tvSeries,Norway,Norwegian,A small Norwegian town experiencing warm winte...,In the small fictional town of Edda coming of ...,7.5,26606.0,"Action,Drama,Fantasy",0,"['David Stakston', 'Jonas Strand Gravli', 'Her...",https://m.media-amazon.com/images/M/MV5BODM3NT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7003,7003,tt9777386,Merry Happy Whatever,"16,543",13,2019.0,NaN,NaN,28,tvEpisode,United States,English,When his daughter arrives home for the holiday...,When his daughter arrives home for the holiday...,6.1,221.0,Comedy,0,"['Dennis Quaid', 'Bridgit Mendler', 'Brent Mor...",https://m.media-amazon.com/images/M/MV5BOTc0ND...
7004,7004,tt9790434,The Morning Show,"16,569",NaN,2019.0,NaN,NaN,30,tvEpisode,-,-,NaN,-,NaN,NaN,\N,0,"['Bounty Hunters Brothers', 'Bruce Lourie', 'C...",https://m.media-amazon.com/images/G/01/imdb/im...
7005,7005,tt9817600,Losers,"16,599",16,2019.0,NaN,NaN,\N,tvEpisode,-,-,"The life of Michael Bentt, a champion boxer fo...","The life of Michael Bentt, a champion boxer fo...",8.0,221.0,"Documentary,Sport",0,"['Michael Bentt', 'Harold Perrineau', 'Mickey ...",https://m.media-amazon.com/images/M/MV5BOTY3Zj...
7006,7006,tt9842922,Firefly Lane,"16,627",18,2021.0,NaN,NaN,51,tvEpisode,United States,English,Kate experiences the force of nature that is T...,Kate experiences the force of nature that is T...,7.3,380.0,"Drama,Romance",0,"['Katherine Heigl', 'Sarah Chalke', 'Ben Lawso...",https://m.media-amazon.com/images/M/MV5BODM3OW...


In [6]:
#number of rows and columns
movie_data.shape

(7008, 20)

In [7]:
#Relevent features for analysis
rel_feature=['genres','cast','title','type','plot']
print(rel_feature)

['genres', 'cast', 'title', 'type', 'plot']


In [8]:
#checking the null values
movie_data.isnull().sum()

index               0
imdb_id             0
title               0
popular_rank        0
certificate      4599
startYear         515
endYear          5882
episodes         4704
runtime             2
type                2
orign_country       0
language            0
plot              399
summary             0
rating           1008
numVotes         1008
genres              2
isAdult             0
cast                0
image_url           0
dtype: int64

In [9]:
#filling the null with '' for the selected features
for feature in rel_feature:
    movie_data[feature]=movie_data[feature].fillna('')
    

In [10]:
#confirm the null valuew for selected feautures are replaced
movie_data.isnull().sum()



index               0
imdb_id             0
title               0
popular_rank        0
certificate      4599
startYear         515
endYear          5882
episodes         4704
runtime             2
type                0
orign_country       0
language            0
plot                0
summary             0
rating           1008
numVotes         1008
genres              0
isAdult             0
cast                0
image_url           0
dtype: int64

In [11]:
#combining all the relevant features
comb_features=movie_data['genres']+' '+movie_data['cast']+' '+ movie_data['title']+ ' '+movie_data['type']+ ' ' +movie_data['plot']
comb_features


0       Crime,Drama,Fantasy ['Tom Ellis', 'Lauren Germ...
1       Action,Crime,Horror ['Dave Bautista', 'Ella Pu...
2       Comedy,Drama ['Michael Douglas', 'Sarah Baker'...
3       Comedy,Romance ['Jennifer Aniston', 'Courteney...
4       Action,Drama,Fantasy ['David Stakston', 'Jonas...
                              ...                        
7003    Comedy ['Dennis Quaid', 'Bridgit Mendler', 'Br...
7004    \N ['Bounty Hunters Brothers', 'Bruce Lourie',...
7005    Documentary,Sport ['Michael Bentt', 'Harold Pe...
7006    Drama,Romance ['Katherine Heigl', 'Sarah Chalk...
7007    Comedy ['Erinn Hayes', 'Rob Huebel', 'Edwin Co...
Length: 7008, dtype: object

In [12]:
#remove unwanted special characters
ch = ["]","["]
for i in ch:
    comb_features=comb_features.str.replace(i, ',')
    
comb_features

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.


0       Crime,Drama,Fantasy ,'Tom Ellis', 'Lauren Germ...
1       Action,Crime,Horror ,'Dave Bautista', 'Ella Pu...
2       Comedy,Drama ,'Michael Douglas', 'Sarah Baker'...
3       Comedy,Romance ,'Jennifer Aniston', 'Courteney...
4       Action,Drama,Fantasy ,'David Stakston', 'Jonas...
                              ...                        
7003    Comedy ,'Dennis Quaid', 'Bridgit Mendler', 'Br...
7004    \N ,'Bounty Hunters Brothers', 'Bruce Lourie',...
7005    Documentary,Sport ,'Michael Bentt', 'Harold Pe...
7006    Drama,Romance ,'Katherine Heigl', 'Sarah Chalk...
7007    Comedy ,'Erinn Hayes', 'Rob Huebel', 'Edwin Co...
Length: 7008, dtype: object

In [13]:
#text to vectors
vector=TfidfVectorizer()


In [14]:
F_vector=vector.fit_transform(comb_features)
print(F_vector)

  (0, 2705)	0.14476643530574262
  (0, 11318)	0.08163108720863585
  (0, 2687)	0.1098295869736332
  (0, 33326)	0.10091656485283639
  (0, 49924)	0.14476643530574262
  (0, 25549)	0.11528189752206634
  (0, 57271)	0.1323216788349423
  (0, 6379)	0.11310298609767641
  (0, 16485)	0.09480135606384214
  (0, 40951)	0.0484677615861809
  (0, 55610)	0.07758925316833419
  (0, 51925)	0.09320833250141332
  (0, 52294)	0.11471188175936686
  (0, 55743)	0.06112170594334913
  (0, 14013)	0.10091656485283639
  (0, 2575)	0.02888399520812099
  (0, 24280)	0.1214391236212404
  (0, 26269)	0.06717766143879551
  (0, 49876)	0.13960139470096744
  (0, 16329)	0.15721119177654286
  (0, 55208)	0.048542415784081985
  (0, 5854)	0.08592861663538821
  (0, 40596)	0.05868362190730797
  (0, 17488)	0.12407124070295067
  (0, 23159)	0.106343712133737
  :	:
  (7007, 24137)	0.15892727108699303
  (7007, 4612)	0.20574214428050955
  (7007, 36978)	0.15411534611784133
  (7007, 22376)	0.11887188185459802
  (7007, 32146)	0.131069420635492
  

In [15]:
#finding similarity score between movies
similarity=cosine_similarity(F_vector)
print(similarity)

[[1.         0.01583218 0.01379883 ... 0.01307554 0.00992016 0.01765683]
 [0.01583218 1.         0.00654593 ... 0.03371039 0.01049915 0.00857472]
 [0.01379883 0.00654593 1.         ... 0.01785242 0.03600459 0.01149756]
 ...
 [0.01307554 0.03371039 0.01785242 ... 1.         0.02515308 0.03221963]
 [0.00992016 0.01049915 0.03600459 ... 0.02515308 1.         0.02353448]
 [0.01765683 0.00857472 0.01149756 ... 0.03221963 0.02353448 1.        ]]


In [16]:
movie_name= input("enter the movie name: ")

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

In [ ]:
allmovie_list=movie_data['title'].tolist()


In [ ]:
#find the close match of the movie that user entered
closest_match=difflib.get_close_matches(movie_name,allmovie_list)
print(closest_match)

In [ ]:
close_match=closest_match[0]
close_match

In [ ]:
#finding the index of the movie
index_movie=movie_data[movie_data.title==close_match]['index'].values[0]
print(index_movie)

In [ ]:
#list of similar movies
similarity_score= list(enumerate(similarity[index_movie]))

In [ ]:
print(similarity_score)

In [ ]:
#sorting the score to get the higher score
high_score=sorted(similarity_score,key=lambda x:x[1],reverse=True)
high_score

In [ ]:
movie=movie_data[movie_data['index']==1]['title'].values[0]
movie

In [ ]:
#suggesting the top 10 movies
print("movies suggested: \n")

i=1
for k in high_score:
    indexx= k[0]
    movie_name_from_index=movie_data[movie_data.index==indexx]['title'].values[0]
    if i<11:
        print(i,'.',movie_name_from_index)
        i+=1
    